In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH14-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH14-Retriever


# ParentDocumentRetriever

1. 개요
    - 문맥이 잘리거나, 청크가 너무 작아 의미 파악이 어려움 
    - 각 청크의 맥락이 유지되도록 충분히 긴 문서를 원하는 경우 많음 
    - 청크 단위로 검색하되, 상위 문서(Parent)를 함께 반환 => 문서 간의 계층 구조 활용 

2. 주요 특징 
    - 검색 효율 + 문맥 보존 동시 확보 
        - 청크 단위로 검색하여 빠르고 정밀한 검색 가능   
        - 결과로는 청크의 전체 원문 문서가 제공되어 문맥 손실 방지  
    - 문서 연결 
        - 청크가 어느 부모 문서에 속하는지 나타내는 Document ID 또는 메타데이터 기반 매핑  
        - LangChain에서는 child -> parent 매핑은 자동 관리 


[Reference] https://python.langchain.com/docs/how_to/parent_document_retriever/

In [3]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [4]:
# 파일 로딩 
loaders = [
    TextLoader("./data/appendix-keywords.txt"),
]

# 문서 로딩 -> docs list 
docs = []
for loader in loaders:
    docs.extend(loader.load())

# 문서 검색

전체 문서를 청크 처리 하지 않고 반영한 경우...

In [5]:
# Child 분할기 정의 
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# DB 정의
vectorstore = Chroma(
    collection_name="full_documents", 
    embedding_function=OpenAIEmbeddings(),
)

# 원본 문서 저장하기 위한 용도
store = InMemoryStore()

# Retriever 정의 
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

문서를 검색기에 추가 

- `ids` : 문서 목록. None이면 자동으로 생성 
- `add_to_docstore=False` : 문서를 중복으로 추가하지 않음 

In [6]:
retriever.add_documents(docs, ids=None, add_to_docstore=True)

In [7]:
# 저장소의 모든 키를 리스트로 반환 (현재는 문서가 1개라 키도 1개가 출력)
list(store.yield_keys())

['c6bdb3fa-6658-4f43-b273-238b7232a79d']

In [8]:
# 유사도 검색
sub_docs = vectorstore.similarity_search("Word2Vec")

In [9]:
# sub_docs 리스트의 첫 번째 요소의 page_content 속성 출력
# 청크 단위 검색
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


In [10]:
# 문서 검색 : Parent 단위 검색
retrieved_docs = retriever.invoke("Word2Vec")

In [11]:
# 검색된 문서의 문서의 페이지 내용의 길이 출력
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서 일부
print(retrieved_docs[0].page_content[2000:2500])

문서의 길이: 5733


 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을


# 청크 적용

- 하나의 원시 문서를 그대로 검색하기는 부적절 
- 청크로 분할할 때, Parent-Child 구분하여 청크 전략 수립 필요 
    - `RecursiveCharacterTextSplitter`를 사용
    - Parent doc : `chunk_size=1000`
    - Child doc : `chunk_size=200` 
- Child는 Parent 보다 작은 크기로 반영 필요 

In [12]:
# Parent
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

# Child
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# VectorDB : Child chunk를 인덱싱으로 사용
vectorstore = Chroma(
    collection_name="split_parents", 
    embedding_function=OpenAIEmbeddings(),
)

# Parent doc 저장
store = InMemoryStore()

# 검색기 정의 
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# 문서 연결
retriever.add_documents(docs) 

In [13]:
# 저장소에서 키를 생성하고 리스트로 변환한 후 길이 반환
len(list(store.yield_keys()))

7

In [14]:
# 유사도 검색
sub_docs = vectorstore.similarity_search("Word2Vec")
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


In [15]:
# 문서 검색
retrieved_docs = retriever.invoke("Word2Vec")
print(retrieved_docs[0].page_content)

정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.
예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.
연관키워드: 딥러닝, 자연어 처리, Attention

HuggingFace

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


-----
** End of Documents **